In [9]:
import pandas as pd

report_list = pd.read_csv('../data/preprocessed/20150101-20221024_삼성전자_d002_list.csv',
                          encoding='cp949',
)
report_list.shape

(709, 4)

In [10]:
report_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업명     709 non-null    object
 1   공시고유번호  709 non-null    int64 
 2   공시날짜    709 non-null    int64 
 3   내용      709 non-null    object
dtypes: int64(2), object(2)
memory usage: 22.3+ KB


In [11]:
report_list.head()

,기업명,공시고유번호,공시날짜,내용
0,삼성전자,20221024000053,20221024,[기재정정]임원ㆍ주요주주특정증권등소유상황보고서
1,삼성전자,20221024000036,20221024,[기재정정]임원ㆍ주요주주특정증권등소유상황보고서
2,삼성전자,20221018000007,20221018,임원ㆍ주요주주특정증권등소유상황보고서
3,삼성전자,20221018000004,20221018,임원ㆍ주요주주특정증권등소유상황보고서
4,삼성전자,20221017000310,20221017,임원ㆍ주요주주특정증권등소유상황보고서


### Download data

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get('DART_API_KEY')

In [13]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = Session()
retry = Retry(connect=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)
session.mount('http://', adapter)

URL = 'https://opendart.fss.or.kr/api/document.xml'

FILEPATH = '../data/origin/dart/reports/d002'

RAW_FILENAME = '{rcept_no}.{extension}'
MY_FILENAME = '{corp_name}_d002_{rcept_no}.{extension}'

In [26]:
def download_report(corp_name:str, rcept_no: str) -> bool:
    # try:
        params = {
            'crtfc_key': API_KEY, 
            'rcept_no': rcept_no
            }
        
        res = session.get(URL, params=params)
        filepath = os.path.join(FILEPATH,
                                MY_FILENAME.format(corp_name=corp_name, rcept_no=rcept_no, extension='zip')
                                )
        with open(filepath, 'wb') as f:
            f.write(res.content)
        
        return True
    # except:
    #     return False

In [25]:
for i, report in report_list.iterrows():
    corp_name = report['기업명']
    rcept_no = report['공시고유번호']
    download_report(corp_name, rcept_no)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## Extract data

In [29]:
import os
import re
import zipfile
from typing import *

MY_PATTERN = r'([^_]+)_d002_(\d+).zip'

def get_name_and_report_code(filename: str) -> Tuple[str, str]:
    name, report_code = re.search(MY_PATTERN, filename).groups()
    return name, report_code
    
def extract_report(name: str, report_code: str)-> Tuple[str, str]:
    zipfile.ZipFile(
        os.path.join(
            FILEPATH,
            MY_FILENAME.format(corp_name=name, rcept_no=report_code, extension='zip')
        )
    )\
        .extractall(FILEPATH)
    
    return name, report_code

def rename_report(name: str, report_code: str):
    os.rename(
        os.path.join(FILEPATH,
                     RAW_FILENAME.format(rcept_no=report_code, extension='xml')
        ),
        os.path.join(FILEPATH,
                     MY_FILENAME.format(corp_name=name, rcept_no=report_code, extension='xml')
        )
    )

In [31]:
filenames = [filename for filename in os.listdir(FILEPATH) if filename[-4:] == '.zip' and filename[:4] == '삼성전자']
error = []

for filename in filenames:
    try:
        name, report_code = get_name_and_report_code(filename)
        extract_report(name, report_code)
        rename_report(name, report_code)
    except:
        error.append((name, report_code))